# Tutorial - build LinearSVC model with sklearn

This tutorial demonstrates how to use the Sci-kit Learn (sklearn) package to build linearSVC model, rank features, and use the model for prediction. We will be using the Kaggle sentiment data again.

Note that sklearn actually provides two SVM algorithms: SVC and LinearSVC. 

The SVC module allows for choosing nonlinear kernels, and it uses one-vs-one strategy for multi-class classification.

The LinearSVC module uses the linear kernel, and it uses one-vs-all strategy for multi-class classification, so linearSVC is generally faster than SVC. Since linear kernel works better for text classification in general, this tutorial demonstrates how to use LinearSVC for text classification.

# Step 1: Read in data

In [1]:
# this step is the same as the NB script

# read in the training data
# the data set includes four columns: PhraseId, SentenceId, Phrase, Sentiment
# In this data set a sentence is further split into phrases 
# in order to build a sentiment classification model
# that can not only predict sentiment of sentences but also shorter phrases

# A data example:
# PhraseId SentenceId Phrase Sentiment
# 1 1 A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .1

# the Phrase column includes the training examples
# the Sentiment column includes the training labels
# "0" for very negative
# "1" for negative
# "2" for neutral
# "3" for positive
# "4" for very positive


import pandas as p
train=p.read_csv("train.tsv", delimiter='\t')
y=train['Sentiment'].values
X=train['Phrase'].values

# Step 2: Split train/test data for hold-out test

In [2]:
# this step is the same as the NB script

# check the sklearn documentation for train_test_split
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
# "test_size" : float, int, None, optional
# If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. 
# If int, represents the absolute number of test samples. 
# If None, the value is set to the complement of the train size. 
# By default, the value is set to 0.25. The default will change in version 0.21. It will remain 0.25 only if train_size is unspecified, otherwise it will complement the specified train_size.    

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])

(93636,) (93636,) (62424,) (62424,)
almost in a class with that of Wilde
3
escape movie
2


Sample output from the code above:

(93636,) (93636,) (62424,) (62424,)
almost in a class with that of Wilde
3
escape movie
2

# Step 2.1 Data Checking

In [3]:
# this step is the same as the NB script

# Check how many training examples in each category
# this is important to see whether the data set is balanced or skewed

training_labels = set(y_train)
print(training_labels)
from scipy.stats import itemfreq
training_category_dist = itemfreq(y_train)
print(training_category_dist)

{0, 1, 2, 3, 4}
[[    0  4141]
 [    1 16449]
 [    2 47718]
 [    3 19859]
 [    4  5469]]


The sample output shows that the data set is skewed with 47718/93636=51% "neutral" examples. All other categories are smaller.

{0, 1, 2, 3, 4}
[[    0  4141]
 [    1 16449]
 [    2 47718]
 [    3 19859]
 [    4  5469]]

# Step 3: Vectorization

In [4]:
# this step is the same as the NB script

# sklearn contains two vectorizers

# CountVectorizer can give you Boolean or TF vectors
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

# TfidfVectorizer can give you TF or TFIDF vectors
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

# Read the sklearn documentation to understand all vectorization options

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# several commonly used vectorizer setting

#  unigram boolean vectorizer, set minimum document frequency to 5
unigram_bool_vectorizer = CountVectorizer(encoding='latin-1', binary=True, min_df=5, stop_words='english')

#  unigram term frequency vectorizer, set minimum document frequency to 5
unigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=5, stop_words='english')

#  unigram and bigram term frequency vectorizer, set minimum document frequency to 5
gram12_count_vectorizer = CountVectorizer(encoding='latin-1', ngram_range=(1,2), min_df=5, stop_words='english')

#  unigram tfidf vectorizer, set minimum document frequency to 5
unigram_tfidf_vectorizer = TfidfVectorizer(encoding='latin-1', use_idf=True, min_df=5, stop_words='english')


## Step 3.1: Vectorize the training data

In [5]:
# this step is the same as the NB script

# The vectorizer can do "fit" and "transform"
# fit is a process to collect unique tokens into the vocabulary
# transform is a process to convert each document to vector based on the vocabulary
# These two processes can be done together using fit_transform(), or used individually: fit() or transform()

# fit vocabulary in training documents and transform the training documents into vectors
X_train_vec = unigram_count_vectorizer.fit_transform(X_train)

# check the content of a document vector
print(X_train_vec.shape)
print(X_train_vec[0].toarray())

# check the size of the constructed vocabulary
print(len(unigram_count_vectorizer.vocabulary_))

# print out the first 10 items in the vocabulary
print(list(unigram_count_vectorizer.vocabulary_.items())[:10])

# check word index in vocabulary
print(unigram_count_vectorizer.vocabulary_.get('imaginative'))

(93636, 11967)
[[0 0 0 ... 0 0 0]]
11967
[('class', 1858), ('wilde', 11742), ('derring', 2802), ('chilling', 1764), ('affecting', 313), ('meanspirited', 6557), ('personal', 7662), ('low', 6296), ('involved', 5602), ('worth', 11868)]
5224


Sample output:

(93636, 11967)
[[0 0 0 ..., 0 0 0]]
11967
[('imaginative', 5224), ('tom', 10809), ('smiling', 9708), ('easy', 3310), ('diversity', 3060), ('impossibly', 5279), ('buy', 1458), ('sentiments', 9305), ('households', 5095), ('deteriorates', 2843)]
5224

## Step 3.2: Vectorize the test data

In [6]:
# this step is the same as the NB script

# use the vocabulary constructed from the training data to vectorize the test data. 
# Therefore, use "transform" only, not "fit_transform", 
# otherwise "fit" would generate a new vocabulary from the test data

X_test_vec = unigram_count_vectorizer.transform(X_test)

# print out #examples and #features in the test set
print(X_test_vec.shape)

(62424, 11967)


Sample output:

(62424, 14324)

# Step 4: Train a LinearSVC classifier

In [17]:
# import the LinearSVC module
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

# initialize the LinearSVC model
svm_clf = LinearSVC(C=1)
svm = SVC()

# use the training data to train the model
svm_clf.fit(X_train_vec,y_train)
svm.fit(X_train_vec,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
svm.

# Step 4.1 Interpret a trained LinearSVC model

In [8]:
## interpreting LinearSVC models
## http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC

## LinearSVC uses a one-vs-all strategy to extend the binary SVM classifier to multi-class problems
## for the Kaggle sentiment classification problem, there are five categories 0,1,2,3,4 with 0 as very negative and 4 very positive
## LinearSVC builds five binary classifier, "very negative vs. others", "negative vs. others", "neutral vs. others", "positive vs. others", "very positive vs. others", 
## and then pick the most confident prediction as the final prediction.

## Linear SVC also ranks all features based on their contribution to distinguish the two concepts in each binary classifier
## For category "0" (very negative), get all features and their weights and sort them in increasing order
feature_ranks = sorted(zip(svm_clf.coef_[0], unigram_count_vectorizer.get_feature_names()))

## get the 10 features that are best indicators of very negative sentiment (they are at the bottom of the ranked list)
very_negative_10 = feature_ranks[-10:]
print("Very negative words")
for i in range(0, len(very_negative_10)):
    print(very_negative_10[i])
print()

## get 10 features that are least relevant to "very negative" sentiment (they are at the top of the ranked list)
not_very_negative_10 = feature_ranks[:10]
print("not very negative words")
for i in range(0, len(not_very_negative_10)):
    print(not_very_negative_10[i])
print()


Very negative words
(1.6216100366141022, 'cesspool')
(1.6484881244856044, 'disappointment')
(1.6592495145108863, 'pompous')
(1.668369722458181, 'stinks')
(1.6927738866942676, 'distasteful')
(1.6955905263376188, 'unwatchable')
(1.7526397248942742, 'unbearable')
(1.7873567535639503, 'stinker')
(1.822870618320237, 'disgusting')
(1.8233056657920867, 'worthless')

not very negative words
(-1.8329269017844116, 'hawke')
(-1.7372806962289808, 'giddy')
(-1.6832953381491251, 'collar')
(-1.5847293662668498, 'swimfan')
(-1.5720764895303911, 'blue')
(-1.4801114078365152, 'dogtown')
(-1.4138361284259322, 'clamoring')
(-1.4093532471911865, 'joan')
(-1.391816224318964, 'victim')
(-1.3400001750134798, 'compulsively')



Sample output:

Very negative words
(1.5635285973159958, 'stunning')
(1.6005795078256047, 'astonish')
(1.610812924790558, 'refreshes')
(1.6148905161082145, 'flawless')
(1.6474647033330083, 'phenomenal')
(1.6506425169734038, 'masterful')
(1.67761558779458, 'masterfully')
(1.8781421016763864, 'glorious')
(1.9801881772566481, 'miraculous')
(2.0143251933052397, 'perfection')

not very negative words
(-2.3147454187985117, 'sacrifices')
(-1.8650987318574794, 'maintained')
(-1.8305667747223913, 'placed')
(-1.7974037295239631, 'argue')
(-1.6800998534753624, '19')
(-1.6684863939524339, 'homage')
(-1.6179084517399509, 'failure')
(-1.6088792786048403, 'breezy')
(-1.6059138072144292, 'bore')
(-1.5466693614369267, 'clone')

# Exercise A

In [9]:
# write code similar to the above sample code 
# and print out the 10 most indicative words for the "very positive" category

# Your code starts here
feature_ranks = sorted(zip(svm_clf.coef_[4], unigram_count_vectorizer.get_feature_names()))

ver_positive_10 = feature_ranks[-10:]
print("Very Positive words")
for i in range(0, len(ver_positive_10)):
    print(ver_positive_10[i])
print()

not_very_pos_10 = feature_ranks[:10]
print("not very positive words")
for i in range(0, len(not_very_pos_10)):
    print(not_very_pos_10[i])
# Your code ends here

Very Positive words
(1.563528542077086, 'stunning')
(1.6005795429764025, 'astonish')
(1.610812826661991, 'refreshes')
(1.6148905318014701, 'flawless')
(1.6474646580587597, 'phenomenal')
(1.6506424963841528, 'masterful')
(1.6776155325392395, 'masterfully')
(1.8781421204829354, 'glorious')
(1.9801881017483018, 'miraculous')
(2.0143251926127466, 'perfection')

not very positive words
(-2.3147453645822136, 'sacrifices')
(-1.8650987489058344, 'maintained')
(-1.8305665512628528, 'placed')
(-1.7974036092615568, 'argue')
(-1.6800998739012027, '19')
(-1.6684864381123057, 'homage')
(-1.61790842418089, 'failure')
(-1.6088792640569234, 'breezy')
(-1.605913808996518, 'bore')
(-1.5466692682887255, 'clone')


Sample output

(1.5635285973159958, 'stunning')
(1.6005795078256047, 'astonish')
(1.610812924790558, 'refreshes')
(1.6148905161082145, 'flawless')
(1.6474647033330083, 'phenomenal')
(1.6506425169734038, 'masterful')
(1.67761558779458, 'masterfully')
(1.8781421016763864, 'glorious')
(1.9801881772566481, 'miraculous')
(2.0143251933052397, 'perfection')

# Step 5: Test the LinearSVC classifier

In [10]:
# test the classifier on the test data set, print accuracy score

svm_clf.score(X_test_vec,y_test)

0.6236864026656415

In [11]:
# print confusion matrix and classification report

from sklearn.metrics import confusion_matrix
y_pred = svm_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[0,1,2,3,4])
print(cm)
print()

from sklearn.metrics import classification_report
target_names = ['0','1','2','3','4']
print(classification_report(y_test, y_pred, target_names=target_names))

[[  918  1221   697    82    13]
 [  701  4080  5504   514    25]
 [  195  2106 27081  2310   172]
 [   34   396  6048  5533  1057]
 [    3    51   590  1772  1321]]

             precision    recall  f1-score   support

          0       0.50      0.31      0.38      2931
          1       0.52      0.38      0.44     10824
          2       0.68      0.85      0.75     31864
          3       0.54      0.42      0.48     13068
          4       0.51      0.35      0.42      3737

avg / total       0.60      0.62      0.60     62424



# Step 5.1 Interpret the prediction result

In [13]:
## get the confidence scores for all test examples from each of the five binary classifiers
svm_confidence_scores = svm_clf.decision_function(X_test_vec)
## get the confidence score for the first test example
print(svm_confidence_scores[0])

svm123 = print(svm_clf.predict_log_proba(X_test_vec))

## sample output: array([-1.05306321, -0.62746206,  0.31074854, -0.89709483, -1.08343089]
## because the confidence score is the highest for category 2, 
## the prediction should be 2. 

## Confirm by printing out the actual prediction
print(y_test[0])

[-1.01718401 -0.50760027  0.22331211 -0.97514725 -1.24718838]


AttributeError: 'LinearSVC' object has no attribute 'predict_log_proba'

# Step 5.2 Error Analysis

In [20]:
# print out specific type of error for further analysis

# print out the very positive examples that are mistakenly predicted as negative
# according to the confusion matrix, there should be 53 such examples
# note if you use a different vectorizer option, your result might be different

err_cnt = 0
for i in range(0, len(y_test)):
    if(y_test[i]==4 and y_pred[i]==0):
        print(X_test[i])
        err_cnt = err_cnt+1
print("errors:", err_cnt)

10 minutes into the film you 'll be white-knuckled and unable to look away .
the film is never dull
greatly impressed by the skill of the actors involved in the enterprise
errors: 3


# Exercise B

In [23]:
# write code to print out 
# the errors that very negative examples were mistakenly predicted as very positive?
# and the errors that very positive examples were mistakenly predicted as very negative?

# Try find lingustic patterns for these two types of errors
# Based on the above error analysis, what suggestions would you give to improve the current model?

# Your code starts here
err_cnt = 0
print(' Very negative < Very positive')
print('---------------------------------')
for i in range(0, len(y_test)):
    if(y_test[i]==0 and y_pred[i]==4):
        print(X_test[i])
        err_cnt = err_cnt+1
print("errors:", err_cnt)

err_cnt = 0
print('Very positive < Very negative:')
print('---------------------------------')
for i in range(0, len(y_test)):
    if(y_test[i]==4 and y_pred[i]==0):
        print(X_test[i])
        err_cnt = err_cnt+1

print("errors", err_cnt)
# Your code ends here

 Very negative < Very positive
---------------------------------
the film is not entirely successful
manages to bleed it almost completely dry of humor , verve and fun .
the spell they cast is n't the least bit mesmerizing
The Master of Disguise is funny -- not
being cast in action films when none of them are ever any good
But if the essence of magic is its make-believe promise of life that soars above the material realm , this is the opposite of a truly magical movie .
is not Edward Burns ' best film
bleed it almost completely dry of humor , verve and fun
Once the expectation of laughter has been quashed by whatever obscenity is at hand , even the funniest idea is n't funny .
manages to bleed it almost completely dry of humor , verve and fun
It may not be a great piece of filmmaking ,
Depicts the sorriest and most sordid of human behavior on the screen , then laughs at how clever it 's being .
a great movie it is not
errors: 13
Very positive < Very negative:
--------------------------

# Step 6: write the prediction output to file

In [19]:
y_pred=svm_clf.predict(X_test_vec)
output = open('/Users/byu/Desktop/data/linearSVC_prediction_output.csv', 'w')
for x, value in enumerate(y_pred):
  output.write(str(value) + '\n') 
output.close()

# Step 6.1 Prepare submission to Kaggle sentiment classification competition

In [21]:
########## submit to Kaggle submission

# we are still using the model trained on 60% of the training data
# you can re-train the model on the entire data set 
#   and use the new model to predict the Kaggle test data
# below is sample code for using a trained model to predict Kaggle test data 
#    and format the prediction output for Kaggle submission

# read in the test data
kaggle_test=p.read_csv("test.tsv", delimiter='\t') 

# preserve the id column of the test examples
kaggle_ids=kaggle_test['PhraseId'].values

# read in the text content of the examples
kaggle_X_test=kaggle_test['Phrase'].values

# vectorize the test examples using the vocabulary fitted from the 60% training data
kaggle_X_test_vec=unigram_count_vectorizer.transform(kaggle_X_test)

# predict using the NB classifier that we built
kaggle_pred=svm_clf.fit(X_train_vec, y_train).predict(kaggle_X_test_vec)

# combine the test example ids with their predictions
kaggle_submission=zip(kaggle_ids, kaggle_pred)

# prepare output file
outf=open('/Users/byu/Desktop/data/kaggle/kaggle_submission_linearSVC.csv', 'w')

# write header
outf.write('PhraseId,Sentiment\n')

# write predictions with ids to the output file
for x, value in enumerate(kaggle_submission): outf.write(str(value[0]) + ',' + str(value[1]) + '\n')

# close the output file
outf.close()

# Exercise C

In [34]:
# generate your Kaggle submissions with boolean representation and TF representation
# submit to Kaggle
# report your scores here
# which model gave better performance in the hold-out test
# which model gave better performance in the Kaggle test


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

#  unigram boolean vectorizer, set minimum document frequency to 5
unigram_bool_vectorizer = CountVectorizer(encoding='latin-1', binary=True, min_df=5, stop_words='english')
#  unigram term frequency vectorizer, set minimum document frequency to 5
unigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=5, stop_words='english')

#Boolean
X_train_vec = unigram_bool_vectorizer.fit_transform(X_train)
X_test_vec = unigram_bool_vectorizer.transform(X_test)

svm_clf = LinearSVC(C=1)
svm_clf.fit(X_train_vec,y_train)
bool_score = svm_clf.score(X_test_vec,y_test) #holdout
print('Holdout test with Boolean Rep. = ', bool_score)

# read in the test data
kaggle_test=p.read_csv("test.tsv", delimiter='\t') 

# preserve the id column of the test examples
kaggle_ids=kaggle_test['PhraseId'].values

# read in the text content of the examples
kaggle_X_test=kaggle_test['Phrase'].values

# vectorize the test examples using the vocabulary fitted from the 60% training data
kaggle_X_test_vec=unigram_bool_vectorizer.transform(kaggle_X_test)

# predict using the NB classifier that we built
kaggle_pred=svm_clf.fit(X_train_vec, y_train).predict(kaggle_X_test_vec)

# combine the test example ids with their predictions
kaggle_submission=zip(kaggle_ids, kaggle_pred)

# prepare output file
outf=open('kaggle_submission_linearSVC_Boolean.csv', 'w')

# write header
outf.write('PhraseId,Sentiment\n')

# write predictions with ids to the output file
for x, value in enumerate(kaggle_submission): outf.write(str(value[0]) + ',' + str(value[1]) + '\n')

# close the output file
outf.close()


# Your code ends here

Holdout test with Boolean Rep. =  0.6241830065359477


In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
from sklearn.feature_extraction.text import TfidfVectorizer

#  unigram boolean vectorizer, set minimum document frequency to 5
unigram_bool_vectorizer = TfidfVectorizer(encoding='latin-1', binary=True, min_df=5, stop_words='english')
#  unigram term frequency vectorizer, set minimum document frequency to 5
tfidf_vectorizer = TfidfVectorizer(encoding='latin-1', min_df=5, stop_words='english')

#Boolean
X_train_vec = tfidf_vectorizer.fit_transform(X_train)
X_test_vec = tfidf_vectorizer.transform(X_test)

svm_clf = LinearSVC(C=1)
svm_clf.fit(X_train_vec,y_train)
bool_score = svm_clf.score(X_test_vec,y_test) #holdout
print('Holdout test with TFIDF Rep. = ', bool_score)

# read in the test data
kaggle_test=p.read_csv("test.tsv", delimiter='\t') 

# preserve the id column of the test examples
kaggle_ids=kaggle_test['PhraseId'].values

# read in the text content of the examples
kaggle_X_test=kaggle_test['Phrase'].values

# vectorize the test examples using the vocabulary fitted from the 60% training data
kaggle_X_test_vec=tfidf_vectorizer.transform(kaggle_X_test)

# predict using the NB classifier that we built
kaggle_pred=svm_clf.fit(X_train_vec, y_train).predict(kaggle_X_test_vec)

# combine the test example ids with their predictions
kaggle_submission=zip(kaggle_ids, kaggle_pred)

# prepare output file
outf=open('kaggle_submission_linearSVC_tfidf.csv', 'w')

# write header
outf.write('PhraseId,Sentiment\n')

# write predictions with ids to the output file
for x, value in enumerate(kaggle_submission): outf.write(str(value[0]) + ',' + str(value[1]) + '\n')

    
# close the output file
outf.close()

Holdout test with TFIDF Rep. =  0.6254325259515571


# Exercise D

In [40]:
# Use the entire Kaggle training data set to build a LinearSVC model
# then do 3-fold CV for the LinearSVC model that you built
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.model_selection import train_test_split
# Your code starts here
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

tfidf_vectorizer = TfidfVectorizer(encoding='latin-1', min_df=5, stop_words='english')
X_train_vec = tfidf_vectorizer.fit_transform(X_train)
X_test_vec = tfidf_vectorizer.transform(X_test)

svm_clf = LinearSVC(C=1)
cv = cross_val_score(svm_clf, X_train_vec, y_train, scoring = "accuracy", cv = 3, n_jobs=4)
import numpy as np
np.mean(cv)


# Your code ends here

In [45]:
import numpy as np
print('Your cv score is:',np.round(np.mean(cv),2))

Your cv score is: 0.61
